In [3]:
import sympy as sym
import sympy.printing as printing

In [32]:
theta, phi, lamb, omega = sym.symbols('Theta Phi Lambda Omega')
alpha, gamma = sym.symbols("alpha gamma")
torque_par, torque_per = sym.symbols("tau_1 tau_2")
Ms, Hoe = sym.symbols("M_s H_{oe}")
energy = sym.Function("U")(theta, phi, lamb, omega)

In [38]:
dUdP = sym.diff(energy, phi)
dUdT = sym.diff(energy, theta)
dUdO = sym.diff(energy, omega)
dUdL = sym.diff(energy, lamb)

sinT = sym.sin(theta)
cosT = sym.cos(theta)

cosL = sym.cos(lamb)
sinL = sym.sin(lamb)

cosPO = sym.cos(phi - omega)
sinPO = sym.sin(phi - omega)
const = (gamma/Ms)*(1/(1+alpha**2))

In [34]:
dTheta = const * ((-1 / sinT)*dUdP - alpha*dUdT -
                  (cosL * sinT - cosT * sinL * cosPO) *
                  (torque_par + alpha * torque_per) - (sinL * sinPO) *
                  (alpha * torque_par - torque_per))

In [35]:
dTheta

gamma*(-alpha*Derivative(U(Theta, Phi, Lambda, Omega), Theta) + (alpha*tau_1 - tau_2)*sin(Lambda)*sin(Omega - Phi) - (alpha*tau_2 + tau_1)*(-sin(Lambda)*cos(Theta)*cos(Omega - Phi) + sin(Theta)*cos(Lambda)) + Derivative(U(Theta, Phi, Lambda, Omega), Phi)/sin(Theta))/(M_s*(alpha**2 + 1))

In [36]:
sym.diff(dTheta, omega)

gamma*(-alpha*Derivative(U(Theta, Phi, Lambda, Omega), Omega, Theta) + (alpha*tau_1 - tau_2)*sin(Lambda)*cos(Omega - Phi) + (-alpha*tau_2 - tau_1)*sin(Lambda)*sin(Omega - Phi)*cos(Theta) + Derivative(U(Theta, Phi, Lambda, Omega), Omega, Phi)/sin(Theta))/(M_s*(alpha**2 + 1))

In [37]:
dPhi = const * ((1 / sinT) * dUdT - (alpha / (sinT**2)) * dUdP + (1 / sinT) *
                (cosL * sinT - cosT * sinL * cosPO) *
                (alpha * torque_par - torque_per) 
                - (1/sinT)*(sinL*sinPO)*(torque_par + alpha*torque_per) + alpha*Ms*Hoe/sinT)
dPhi

gamma*(H_{oe}*M_s*alpha/sin(Theta) - alpha*Derivative(U(Theta, Phi, Lambda, Omega), Phi)/sin(Theta)**2 + (alpha*tau_1 - tau_2)*(-sin(Lambda)*cos(Theta)*cos(Omega - Phi) + sin(Theta)*cos(Lambda))/sin(Theta) + (alpha*tau_2 + tau_1)*sin(Lambda)*sin(Omega - Phi)/sin(Theta) + Derivative(U(Theta, Phi, Lambda, Omega), Theta)/sin(Theta))/(M_s*(alpha**2 + 1))

In [39]:
dLam = const * ((-1 / sinT)*dUdO - alpha*dUdL +
                  (-sinL * cosT + sinT * cosL * cosPO) *
                  (torque_par + alpha * torque_per) + (sinL * sinPO) *
                  (alpha * torque_par + torque_per) + Ms*Hoe) 
dLam

gamma*(H_{oe}*M_s - alpha*Derivative(U(Theta, Phi, Lambda, Omega), Lambda) - (alpha*tau_1 + tau_2)*sin(Lambda)*sin(Omega - Phi) + (alpha*tau_2 + tau_1)*(-sin(Lambda)*cos(Theta) + sin(Theta)*cos(Lambda)*cos(Omega - Phi)) - Derivative(U(Theta, Phi, Lambda, Omega), Omega)/sin(Theta))/(M_s*(alpha**2 + 1))

In [42]:
dOmega = const * ((1 / sinT) * dUdL - (alpha / (sinL**2)) * dUdO - (1 / sinL) *
                (sinL * cosT - sinT *cosL * cosPO) *
                (torque_per - alpha * torque_par) 
                + (1/sinL)*(sinT*sinPO)*(torque_par - alpha*torque_per) + alpha*Ms*Hoe/sinL)
dOmega

gamma*(H_{oe}*M_s*alpha/sin(Lambda) - alpha*Derivative(U(Theta, Phi, Lambda, Omega), Omega)/sin(Lambda)**2 - (-alpha*tau_1 + tau_2)*(sin(Lambda)*cos(Theta) - sin(Theta)*cos(Lambda)*cos(Omega - Phi))/sin(Lambda) - (-alpha*tau_2 + tau_1)*sin(Theta)*sin(Omega - Phi)/sin(Lambda) + Derivative(U(Theta, Phi, Lambda, Omega), Lambda)/sin(Theta))/(M_s*(alpha**2 + 1))

In [44]:
dM = sym.Matrix([
    [
        sym.diff(dTheta, theta),
        sym.diff(dTheta, phi),
        sym.diff(dTheta, lamb),
        sym.diff(dTheta, omega)
    ],
    [
        sym.diff(dPhi, theta),
        sym.diff(dPhi, phi),
        sym.diff(dPhi, lamb),
        sym.diff(dPhi, omega)
    ],
    [
        sym.diff(dLam, theta),
        sym.diff(dLam, phi),
        sym.diff(dLam, lamb),
        sym.diff(dLam, omega)
    ],
    [
        sym.diff(dOmega, theta),
        sym.diff(dOmega, phi),
        sym.diff(dOmega, lamb),
        sym.diff(dOmega, omega)
    ]]
)

In [45]:
dM

Matrix([
[                                                                                                                                                                                                                                                                                                                                                          gamma*(-alpha*Derivative(U(Theta, Phi, Lambda, Omega), (Theta, 2)) + (-alpha*tau_2 - tau_1)*(sin(Lambda)*sin(Theta)*cos(Omega - Phi) + cos(Lambda)*cos(Theta)) + Derivative(U(Theta, Phi, Lambda, Omega), Phi, Theta)/sin(Theta) - cos(Theta)*Derivative(U(Theta, Phi, Lambda, Omega), Phi)/sin(Theta)**2)/(M_s*(alpha**2 + 1)),                                           gamma*(-alpha*Derivative(U(Theta, Phi, Lambda, Omega), Phi, Theta) - (alpha*tau_1 - tau_2)*sin(Lambda)*cos(Omega - Phi) - (-alpha*tau_2 - tau_1)*sin(Lambda)*sin(Omega - Phi)*cos(Theta) + Derivative(U(Theta, Phi, Lambda, Omega), (Phi, 2))/sin(Theta))/(M_s*(alpha**2 + 1)),          

In [46]:
dM.eigenvals()

KeyboardInterrupt: 